In [86]:
import pandas as pd
games_details=pd.read_csv('games_details.csv', low_memory=False)
players=pd.read_csv('players.csv')
games=pd.read_csv('games.csv')

print(games.isna().sum())
print(games_details.isna().sum())
print(players.isna().sum())
len(games_details)

GAME_DATE_EST        0
GAME_ID              0
GAME_STATUS_TEXT     0
HOME_TEAM_ID         0
VISITOR_TEAM_ID      0
SEASON               0
TEAM_ID_home         0
PTS_home            99
FG_PCT_home         99
FT_PCT_home         99
FG3_PCT_home        99
AST_home            99
REB_home            99
TEAM_ID_away         0
PTS_away            99
FG_PCT_away         99
FT_PCT_away         99
FG3_PCT_away        99
AST_away            99
REB_away            99
HOME_TEAM_WINS       0
dtype: int64
GAME_ID                   0
TEAM_ID                   0
TEAM_ABBREVIATION         0
TEAM_CITY                 0
PLAYER_ID                 0
PLAYER_NAME               0
NICKNAME             615591
START_POSITION       412863
COMMENT              558939
MIN                  109690
FGM                  109690
FGA                  109690
FG_PCT               109690
FG3M                 109690
FG3A                 109690
FG3_PCT              109690
FTM                  109690
FTA                  109690


668628

In [87]:
# subsetting
required_columns = ['GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'MIN', 'FGM', 'FTM', 'TO', 'PTS', 'REB', 'AST', 'STL', 'BLK']


games_details_subset = games_details[required_columns]

In [88]:
games.dropna(inplace=True)

players.dropna(inplace=True)

In [89]:
games_details_subset.dropna(inplace=True)
len(games_details)



<ipython-input-89-5371eff29aeb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_details_subset.dropna(inplace=True)


668628

In [90]:
import pandas as pd



# Select subset of required columns from games_details
required_columns = ['GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'MIN', 'FGM', 'FTM', 'TO', 'PTS', 'REB', 'AST', 'STL', 'BLK']
games_details_subset = games_details[required_columns]

# Handling missing values in each dataset
games.dropna(inplace=True)
games_details_subset.dropna(inplace=True)
players.dropna(inplace=True)

# Merging datasets
merged_data = pd.merge(games, games_details_subset, left_on='GAME_ID', right_on='GAME_ID')
merged_data = pd.merge(merged_data, players, left_on='PLAYER_ID', right_on='PLAYER_ID')




<ipython-input-90-714b680c817c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_details_subset.dropna(inplace=True)


In [91]:

# Converting 'MIN' column to numeric values
merged_data['MIN'] = pd.to_numeric(merged_data['MIN'], errors='coerce')

# Droping rows where 'MIN' is NaN
merged_data = merged_data.dropna(subset=['MIN'])

# Calculate PER for each player
merged_data['PER'] = (
    (merged_data['PTS'] + merged_data['REB'] + merged_data['AST'] + merged_data['STL'] + merged_data['BLK']
     - merged_data['FGM'] - merged_data['FTM'] - merged_data['TO'])
    / merged_data['MIN']
) * 48  # Per-minute multiplier

# Group by player, team, and season to analyze PER across teams and seasons
per_analysis = merged_data.groupby(['PLAYER_NAME', 'TEAM_ID_home', 'SEASON_x'])['PER'].mean().reset_index()

#most efficient players
most_efficient_players = per_analysis.loc[per_analysis.groupby(['SEASON_x', 'TEAM_ID_home'])['PER'].idxmax()]

# Display the results
print("Player Efficiency Ratings (PER) Analysis:")
print(most_efficient_players)
# top 10 efficient players
#top_10_efficient_players = per_analysis.groupby(['SEASON_x', 'TEAM_ID_home']).apply(lambda x: x.nlargest(10, 'PER')).reset_index(drop=True)


#print("Top 10 Player Efficiency Ratings (PER) for each SEASON and TEAM_ID_home:")
#print(top_10_efficient_players)





Player Efficiency Ratings (PER) Analysis:
            PLAYER_NAME  TEAM_ID_home  SEASON_x         PER
3053       Darius Miles    1610612737      2003  192.000000
1193        Ben Wallace    1610612738      2003  172.800000
3941        Drew Gooden    1610612740      2003  201.600000
12138        Steve Nash    1610612742      2003  163.200000
4027        Dwyane Wade    1610612748      2003  240.000000
...                 ...           ...       ...         ...
3305     DeAndre Jordan    1610612762      2013   39.272727
4020      Dwight Howard    1610612763      2013   41.600000
7451      Kevin Garnett    1610612764      2013   40.000000
7144    Justin Hamilton    1610612765      2013   42.666667
10126  Patrick O'Bryant    1610612766      2013   64.000000

[311 rows x 4 columns]


<ipython-input-91-9dd7854a00a0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['PER'] = (


In [74]:
# Find the index of the maximum PER for each unique year and team combination
max_per_index = merged_data.groupby(['SEASON_x', 'TEAM_ID_home'])['PER'].idxmax()


highest_per_players = merged_data.loc[max_per_index]

# Drop duplicate years, keeping only the player with the maximum total points and total rebounds in case of ties
highest_per_players = highest_per_players.sort_values(by=['SEASON_x', 'TEAM_ID_home', 'PTS', 'REB'], ascending=[True, True, False, False])
highest_per_players = highest_per_players.drop_duplicates(subset=['SEASON_x', 'TEAM_ID_home'])


print("Highest PER Player for Each Year:")
print(highest_per_players[['SEASON_x', 'TEAM_ID_home', 'PLAYER_NAME', 'PER', 'PTS', 'REB']])

Highest PER Player for Each Year:
         SEASON_x  TEAM_ID_home       PLAYER_NAME         PER   PTS   REB
3561750      2003    1610612737      Darius Miles  192.000000  13.0   7.0
3479369      2003    1610612738       Ben Wallace  172.800000  17.0   6.0
2133719      2003    1610612740       Drew Gooden  201.600000  29.0   5.0
3027833      2003    1610612742        Steve Nash  163.200000  12.0   2.0
1908239      2003    1610612748       Dwyane Wade  240.000000  18.0   8.0
...           ...           ...               ...         ...   ...   ...
1060693      2013    1610612762    Derrick Favors   40.320000  10.0  14.0
1270357      2013    1610612763     Dwight Howard   41.600000   6.0  10.0
3265422      2013    1610612764    Josh Childress   64.000000   3.0   1.0
2831569      2013    1610612765   Justin Hamilton   42.666667   8.0   4.0
3288030      2013    1610612766  Patrick O'Bryant   64.000000   2.0   5.0

[311 rows x 6 columns]


In [85]:
a=most_efficient_players.to_csv('/content/sample_data/most_efficient_players.csv', index=False)